In [2]:
import pandas as pd
from sqlalchemy import create_engine
import yfinance as yf
from datetime import date

In [ ]:
user = "root"
password = "fBTJ3_a_3Rg^V!K"
host = "localhost"     # or server IP
database = "stockprice_db"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

def sql_push(df,table_name):
    df.to_sql(name = table_name, con = engine, if_exists = "replace", index = False)

def download_push(ticker, tab_name):
    symbol = yf.Ticker('IOC.NS')
    today = date.today
    iocl_df = symbol.history(start= "2007-04-01", actions = True)
    sdf = yf.download(ticker, start= today.replace(year = today.year - 10))
    sdf.reset_index(inplace=True)
    sdf = pd.DataFrame(sdf)
    sql_push(sdf, tab_name)

In [7]:
download_push(ticker="IOC.NS", tab_name="iocl")

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [11]:
input("Run the query in 'ioc table prep.sql' file in MySQL, once done Press Enter to Continue ")
log_df = pd.read_sql("SELECT date, close, log_close FROM iocl_clean", engine)
print(log_df.head())

         date  close  log_close
0  2015-11-23  36.99       3.61
1  2015-11-24  37.59       3.63
2  2015-11-25  37.59       3.63
3  2015-11-26  37.82       3.63
4  2015-11-27  37.68       3.63


In [12]:
def index_date(df):
    rdf = df.copy()
    rdf['date'] = pd.to_datetime(df['date'])
    rdf.set_index('date', inplace = True)
    return rdf

In [13]:
stock_df = index_date(log_df)
print(stock_df.head())

            close  log_close
date                        
2015-11-23  36.99       3.61
2015-11-24  37.59       3.63
2015-11-25  37.59       3.63
2015-11-26  37.82       3.63
2015-11-27  37.68       3.63
